In [140]:
import collections
import itertools
import functools
import math
# import multiprocessing
import operator
import string
# import random

import numba
import numpy as np
# import pandas as pd

from nltk import tokenize
from nltk.util import ngrams

# from scipy.spatial.distance import jaccard
# from scipy.special import expit as sigmoid

# from sklearn.metrics import jaccard_similarity_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

In [141]:
import json
import pathlib

cwd = pathlib.Path.cwd()
data = cwd / 'data'
jsons = data / 'jsons'
json_2018 = jsons / '2018'

json_2018 = list(json_2018.iterdir())[0]

In [150]:
with open(json_2018) as fp:
    articles_2018 = json.load(fp)['2018']

article = articles_2018[4]
text = article['story']
summ_true = article['summary']
# print(text)

# Evolutionary Algorithms
- [Discrete Differential Evolution for Text Summarization](https://www.researchgate.net/publication/281662415_Discrete_Differential_Evolution_for_Text_Summarization)
- [Evolutionary Algorithm for Extractive Text Summarization](https://www.researchgate.net/profile/Ramiz_Aliguliyev/publication/220518077_Evolutionary_Algorithm_for_Extractive_Text_Summarization/links/09e4151356fc2caab6000000.pdf)
- [An Improved Evolutionary Algorithm for Extractive Text Summarization](https://link.springer.com/chapter/10.1007/978-3-642-36543-0_9)

## Sentence Clustering

In [4]:
def distinct_words(text):
    no_punctuation = ''.join(t for t in text if t not in string.punctuation)
    return frozenset(tokenize.word_tokenize(no_punctuation))

Let document $ D $ be decomposed into a set of $ n $ sentences.  
$
D = \{ S_1, S_2, ..., S_n \}
$

In [5]:
D = document_sentences = set(tokenize.sent_tokenize(text))

Let terms $ T $ be the set of all $ m $ distinct words in $ D $.  
$
T = \{ t_1, t_2, ..., t_m \}
$

In [6]:
T = document_distinct_words = distinct_words(text)

Let $ S_i $ represent the set of distinct terms in sentence $ S_i $ with 
$ m_i $ distinct terms.  
$
S_i = \{ t_1, t_2, ..., t_{ m_i } \}
$

In [7]:
S = sentence_distinct_words = {distinct_words(ds) for ds in document_sentences}

## Similarity Measures

### Jaccard Coefficient Similarity Measure

$
\large
sim_{ jaccard } ( S_i, S_j ) = 
\dfrac
    { | S_i \cap S_j | }
    { | S_i \cup S_j | }
$

### Normalized Google Metrics

$
\large
\text{NGD} ( t_k, t_l ) = \dfrac 
    { \text{max} \big\{
        \text{log} ( f_k ), 
        \text{log} ( f_l )
    \big\}
    - \text{log} ( f_{ lk } ) }
    { \text{log} ( n ) 
    - \text{min} \big\{
        \text{log} ( f_k ), 
        \text{log} ( f_l )
    \big\} }
$

where:
- $ t_k $ and $ t_l $ are terms 
- $ f_k $ is the number of sentences containing $ t_k $
- $ f_{ kl } $ is the number of sentences containing both $ t_k $ and $ t_l $
- $ n $ is the total number of sentences

$
\large 
sim_{ \text{NGD} } ( t_k, t_l ) = \text{exp} 
    \big( - \text{NGD} ( t_k, t_l ) \big)
$

$
\large 
sim_{ \text{NGD} } ( S_i, S_j ) = 
\dfrac
    { \sum\limits
        _{ \small t_k \in S_i } 
        \sum\limits
            _{ \small t_l \in S_j } 
            sim_{ \text{NGD} } ( t_k, t_l ) }
    { m_i m_j }
$  

where:
- $ S_i $ and $ S_j $ are sentences
- $ m_i $ is the number of words in $ S_i $

In [8]:
class NormalizedGoogle:
    def __init__(self, document):
        self.sentence_words = tuple(distinct_words(sent) for sent in tokenize.sent_tokenize(document))
        
    # double check scientific paper's handling of "bad" log values
    def distance(self, term_k, term_l):
        freq_k = sum(term_k in sent for sent in self.sentence_words)
        freq_l = sum(term_l in sent for sent in self.sentence_words)
        if not (freq_k and freq_l):
            raise ValueError('terms must be in document')

        freq_kl = sum((term_k in sent) and (term_l in sent) for sent in self.sentence_words)
        if (freq_k > 0) and (freq_l > 0) and (freq_kl == 0):
            return 1.0

        logs_k_l = (math.log(freq_k), math.log(freq_l))
        n = len(self.sentence_words)

        numerator = max(logs_k_l) - math.log(freq_kl)
        denominator = math.log(n) - min(logs_k_l)
        return numerator / denominator
    
    def term_similarity(self, term_k, term_l):
        dist = self.distance(term_k, term_l)
        return math.exp(-dist)
    
    def sentence_similarity(self, sent_i, sent_j):
        total = sum(self.term_similarity(term_k, term_l)
                    for term_k, term_l in itertools.product(sent_i, sent_j))
        return total / len(sent_i) / len(sent_j)

## Objective Functions

Let $ C $ be a partition of $ D $ with $ k $ clusters.  
$ C = \{ C_1, C_2, ..., C_k \} $  

where:
- $ C_p \cap C_q = \emptyset, 
        \forall p \ne q \in \{ 1, 2, ..., k \} 
  $
- $ \bigcup\limits
    _{ p = 1 }
    ^k C_p = D 
  $
- $ C_p \ne \emptyset, 
        \forall p \in \{ 1, 2, ..., k \}
  $

In [9]:
def verify_clusters(clusters, document):
    disjoint = all(not cluster_i & cluster_j for cluster_i, cluster_j in itertools.combinations(clusters, r=2))
    union = functools.reduce(operator.or_, clusters) == document
    nonempty = all(cluster for cluster in clusters)
    if not (disjoint and union and nonempty):
        raise ValueError('clusters do not form a partition')

In [10]:
def clusterize(chromosome):
    partition = collections.defaultdict(list)
    for i, cluster in enumerate(chromosome):
        partition[cluster].append(i)
    return partition

### Sigmoid Function
$
\large
sigm ( x ) = 
\dfrac
    { 1 }
    { 1 + \text{exp} ( -x ) }
$

In [11]:
from scipy.special import expit as sigmoid
# interesting this is 3x faster than my implimentation

### Intra-Cluster Similiarity (Cohesion)

$
\large
F_1 = 
\sum\limits
    _{ \small p = 1 }
    ^{ \small k } 
\sum\limits
    _{ \small S_i, S_j \in C_p } 
\dfrac 
    { sim ( S_i, S_j ) } 
    { | C_p | } 
\rightarrow \text{max}
$

*(__Note:__ Evol Alg for Ext Txt Summ doesn't show division of $|C_p|$ but I believe this to be a typo since the paragraph detailing it says "the average sum". Additionally the Disc Diff Evol for Txt Summ shows it as such.)*

### Inter-Cluster Dissimilarity (Separation)

$
\large
F_2 = 
\sum\limits
    _{ \small p = 1 }
    ^{ \small k - 1 }
\sum\limits
    _{ \small q = p + 1 }
    ^{ \small k } 
\sum\limits
    _{ \small S_i \in C_p } 
\sum\limits
    _{ \small S_j \in C_q } 
\dfrac 
    { sim ( S_i, S_j ) } 
    { | C_p | \cdot | C_q | }
\rightarrow \text{min}
$

### Inter/Intra-Cluster Balance

$
\large
F = \big( 1 + sigm ( F_1 ) \big)
    ^{ F_2 } \rightarrow \text{max}
$

In [12]:
cv = CountVectorizer()
vec = cv.fit_transform(tokenize.sent_tokenize(text))

doc = vec.toarray().astype(bool).astype(int)
chrom = np.array([0, 2, 1, 1, 3, 2, 4, 1])


def cohesion(chromosome, sim, doc):
    total = 0
    for p in np.unique(chromosome):
        cluster_p = doc[np.where(chromosome == p)]
        for sent_i, sent_j in itertools.combinations(cluster_p, r=2):
            total += sim(sent_i, sent_j) / len(cluster_p)
    return total


def separation(chromosome, sim, doc):
    total = 0
    for p, q in itertools.combinations(np.unique(chromosome), r=2):
        cluster_p = doc[np.where(chromosome == p)]
        cluster_q = doc[np.where(chromosome == q)]
        for sent_i, sent_j in itertools.product(cluster_p, cluster_q):
            total += sim(sent_i, sent_j) / len(cluster_p) / len(cluster_q)
    return total

def cohesion_separation(chromosome, sim, doc):
    coh = cohesion(chromosome, sim, doc)
    sep = separation(chromosome, sim, doc)
    return pow(1 + sigmoid(coh), sep)

In [171]:
@numba.njit
def jaccard_sim(a, b):
    #: assume union is non-empty since each sentence >= 1 word
    return np.sum(a & b) / np.sum(a | b)


@numba.njit
def sigmoid2(x):
    return 1 / (1 + np.exp(-x))


@numba.njit
def cohesion2(chromosome, sim, doc):
    total = 0
    for p in np.unique(chromosome):
        sents = doc[np.where(chromosome == p)]
        k = len(sents)
        #: combinations choose 2
        for i in range(k-1):
            for j in range(i+1, k):
                total += sim(sents[i], sents[j]) / len(sents)  
    return total



@numba.njit
def separation2(chromosome, sim, doc):
    total = 0
    k = len(np.unique(chromosome))
    #: combinations choose 2
    for p in range(k-1):
        for q in range(p+1, k):
            sents_p = doc[np.where(chromosome == p)]
            sents_q = doc[np.where(chromosome == q)]
            #: product
            m, n = len(sents_p), len(sents_q)
            for i in range(m):
                for j in range(n):
                    total += sim(sents_p[i], sents_q[j]) / m / n
    return total


@numba.njit
def cohesion_separation2(chromosome, sim, doc):
    coh = cohesion2(chromosome, sim, doc)
    sep = separation2(chromosome, sim, doc)
    return (1 + sigmoid2(coh)) ** sep



cv = CountVectorizer(stop_words='english')
sents = tokenize.sent_tokenize(text.lower())
vec = cv.fit_transform(sents)

doc = vec.toarray().astype(bool).astype(int)
chrom = np.array([0, 2, 1, 1, 3, 2, 4, 1])

assert cohesion2(chrom, jaccard_sim, doc) == cohesion(chrom, jaccard_sim, doc)
assert separation2(chrom, jaccard_sim, doc) == separation(chrom, jaccard_sim, doc)
assert cohesion_separation2(chrom, jaccard_sim, doc) == cohesion_separation(chrom, jaccard_sim, doc)

## Fitness

$
\large
fitness_1 \big( X_a ( t ) \big) = F_1 \big( X_a ( t ) \big)
$

$
\large
fitness_2 \big( X_a ( t ) \big) = \dfrac{ 1 }{ F_2 ( X_a ( t ) ) }
$

$
\large
fitness \big( X_a ( t ) \big) = F \big( X_a ( t ) \big)
$

# Modified Discrete Differential Evolution Algorithm

Initialize the population with $ N $ chromosomes each composed of $ n $ random integers from \[1, k\].  

$
X_r ( t ) = [ x_{ r, 1 } ( t ), x_{ r, 2 } ( t ), ..., x_{ r, n } ( t ) ]
$  

where:
- $ x_{ r, s } ( t ) \in \{ 1, 2, ..., k \} $
- $ r = 1, 2, ..., N $
- $ s = 1, 2, ..., n $
- $ N $ is the population size
- $ n $ is the number of sentences _(in the document)_
- $ k $ is the number of clusters _(number of sentences for summary)_
- $ t $ is the iteration step

$
\large
y_{ r, s } ( t + 1 ) = 
\begin{cases}
    x_{ r1, s } ( t ) + \lambda \big( x_{ r2, s } ( t ) - x_{ r3, s } ( t ) \big) 
        & \text{if } rand_s < \text{CR} \\
    x_{ r, s } ( t ) & \text{otherwise}
\end{cases}
$

where  
- For each $ X_r $, randomly sample $ X_{ r1 } ( t ), X_{ r2 } ( t ), X_{ r3 } ( t ) $ 
  from the same generation _(each distinct)_
- $ rand_s $ is uniformally distributed random numbers from $ [ 0, 1 ] $ 
  chosen once for each $ s \in \{ 1, 2, ..., n \} $

hyper-parameters 
- $ \lambda $ is a scale factor from $ [ 0, 1 ] $
- $ \text{CR} $ is the crossover rate from $ [ 0, 1 ] $

In [229]:
def init_chromosome(choices, length):
    chrom = np.full(length, -1)
    #: ensure that each choice is accounted for at least once
    idxs = np.random.choice(np.arange(length), len(choices), replace=False)
    chrom[idxs] = np.random.permutation(choices)
    idxs = np.where(chrom == -1)[0]
    chrom[idxs] = np.random.choice(choices, len(idxs))
    return chrom


def init_population(pop_size, clust_amt, chrom_len):
    clusts = np.arange(clust_amt)
    chroms = [init_chromosome(clusts, chrom_len) for _ in range(pop_size)]
    pop = np.vstack(chroms)
    return pop
    

def get_offspring(pop, rand, lam, cr):
    #: For computation time, relax requirement that X_r, X_r1, X_r2, X_r3 are distinct. 
    #: With large population size, this is unlikely to occur, and if it does, it doesn't
    #: seem that detrimental. Also is this mitigated with appropriate lam choice?
    n = len(pop)
    idxs = np.random.choice(np.arange(n), size=(n, 3))
    chrom_1, chrom_2, chrom_3 = map(np.squeeze, np.split(pop[idxs], 3, axis=1))
    k = len(np.unique(pop))
    offspr = (chrom_1 + lam * (chrom_2 - chrom_3)) % k
    mask = rand < cr
    offspr[mask] = pop[mask]
    return offspr


# @numba.njit(parallel=True)
# def get_mask(pop, offspr, func):
#     fit_pop = np.empty_like(pop)
#     fit_off = np.empty_like(offspr)
#     for i in numba.prange(len(pop)):
#         fit_pop[i] = func(pop[i])
#         fit_off[i] = func(offspr[i])

#     mask = fit_off > fit_pop
#     return mask

# def next_generation(pop, offspr, func):
#     mask = get_mask(pop, offspr, func)
#     pop[mask] = offspr[mask]
#     return


def next_generation(pop, offspr, func):
    fit_off = np.array([func(chrom) for chrom in offspr])
    fit_pop = np.array([func(chrom) for chrom in pop])
    mask = fit_off > fit_pop
    pop[mask] = offspr[mask]
    return


# def next_generation(pop, offspr, func):
#     fit_off = func(offspr)
#     fit_pop = func(pop)
#     mask = fit_off > fit_pop
#     pop[mask] = offspr[mask]
#     return


def mutate(pop, rand):
    mask = rand < sigmoid(pop)
    idxs = np.nonzero(mask)
    #: reverse indices for each row
    rev = np.vstack(idxs).T
    rev = np.array(sorted(rev, key=lambda x: (x[0], -x[1])))
    rev = tuple(np.split(rev.T, 2))
    pop[idxs] = pop[rev]
    return


#TODO: early stopping --> little fitness improvement over x generations, good enough fitness score
def run_iterations(pop_size, summ_len, num_sents, func, lam, cr, iterations, *, mutate_after=True,
                   seed=None, verbose=False, save_rate=np.nan, save_dir=None):
    
    if save_dir is not None:
        save_dir = pathlib.Path(save_dir)
        if not save_dir.is_dir():
            msg = f'save_dir={save_dir} not a valid directory path'.format(save_dir=save_dir)
            raise NotADirectoryError(msg)
    
    if seed is not None:
        np.random.seed(seed)
    
    pop = init_population(pop_size, summ_len, num_sents)
    shape = pop.shape
    for i in range(iterations):
        if i % save_rate == 0:
            file = save_dir / 'generation_{i:0>pad}'.format(i=i, pad=len(str(iterations)))
            np.save(file, pop)
            
        if verbose:
            print(i)  #TODO: logfile --> iteration number, best fitness score, avg fitness score, hyper-params
        
        rand = np.random.random_sample(shape)
        
        t0 = time.time()
        offspr = get_offspring(pop, rand, lam, cr)
        t1 = time.time()
        PROFILER['offspring'] += t1 - t0
        
        #: option since papers unclear if mutate at offspring or survivors stage
        if not mutate_after:
            mutate(offspr, rand)
            
        t0 = time.time()
        next_generation(pop, offspr, func)
        t1 = time.time()
        PROFILER['generation'] += t1 - t0
        
        if mutate_after:
            t0 = time.time()
            mutate(pop, rand)
            t1 = time.time()
            PROFILER['mutate'] += t1 - t0
    
    return pop

In [230]:
import time


scaling_factor = 0.9
crossover_rate = 0.5

PROFILER = collections.Counter()

@numba.njit
def fitness(chromosome):
    return cohesion_separation2(chromosome, jaccard_sim, doc)

t0 = time.time()
pop = run_iterations(pop_size=100, summ_len=5, num_sents=len(doc), 
                     func=fitness, lam=0.9, cr=0.5, iterations=200, verbose=False, seed=0)
t1 = time.time()
print(t1-t0)

14.08823299407959


In [231]:
PROFILER

Counter({'offspring': 0.053069114685058594,
         'generation': 13.132251739501953,
         'mutate': 0.8798696994781494})

In [175]:
def best_chromosome(population):
    #TODO: make sure it picks one with all k-clusters
    fits = np.argmax([fitness(chrom) for chrom in population])
    chrom = population[fits]
    return chrom
    

def central_sentences(chromosome, document, metric=cosine_distances):
    central_sents = []
    for cluster in np.unique(chromosome):
        idxs = np.where(chromosome == cluster)[0]
        sents = document[idxs]
        centroid = sents.mean(axis=0)[np.newaxis,:]
        dists = metric(sents, centroid)
        cent_sent = idxs[np.argmin(dists)]
        central_sents.append(cent_sent)
    return central_sents


chrom_best = best_chromosome(pop)
idxs = central_sentences(chrom_best, doc)
summ_evol = '\n'.join(np.array(tokenize.sent_tokenize(text))[idxs])

$
\large
X_r(t+1) = 
\begin{cases}
    Y_r ( t + 1 ) & \text{if } f \big( Y_r ( t + 1 ) \big) > f \big( X_r ( t ) \big) \\
    X_r ( t ) & \text{otherwise}
\end{cases}
$

where 
- $ f ( \cdot ) $ is the objective function to be maximized

## Mutation

At each iteration $ t + 1 $ for each $ X_r ( t ) $ creates
$ m_r ( t + 1 ) = [ m_{ r, 1 } ( t ), m_{ r, 2 } ( t ), ..., m_{ r, n } ( t ) ] $.  
For each gene, 1 indicates no mutation and 0 means mutate.

$
\large
m_{ r, s } ( t + 1 ) = 
\begin{cases}
    1 & \text{if } rand_s < sigm \big( y_{ r, s } ( t + 1 ) \big) \\
    0 & \text{otherwise}
\end{cases}
$

## Inversion Operator

### Figure 1 - Psuedo-code
<img src="./data/pngs/fig1_-_inversion_operator_psuedo_code.png" alt="Fig 1. Inverse operator psuedo-code" width="33%" align="left"/>

### Figure 2 - Example
<img src="./data/pngs/fig2_-_inversion_operator_diagram.png" alt="Fig 2. Inverse operator example" width="33%" align="left"/>

## ROUGE-N
_(Recall Oriented Understudy for Gisting Evaluation)_

$
\large
\text{ROUGE-N} = 
\dfrac
    { \sum \limits
        _{ \small S \in Summ_{ ref }} 
        \sum \limits
        _{ \small \text{N-gram} \in S } 
            Count_{ \small match } ( \text{N-gram} ) }
    { \sum \limits
        _{ \small S \in Summ_{ ref } } 
        \sum \limits
            _{ \small \text{N-gram} \in S } 
            Count( \text{N-gram} ) }
$

In [176]:
def rouge_n(n, y_pred, y_true):
    n_gram_pred = set(ngrams(y_pred, n))
    n_gram_true = set(ngrams(y_true, n))
    return len(n_gram_pred & n_gram_true) / len(n_gram_true)


rouge_n = functools.partial(rouge_n, y_true=summ_true)


def scores(n, y_pred):
    for i in range(1, n+1):
        score = rouge_n(i, y_pred)
        print(i, score)

In [188]:
print('evolutionary')
scores(5, summ_evol)

evolutionary
1 0.9230769230769231
2 0.8529411764705882
3 0.5827338129496403
4 0.46938775510204084
5 0.3959731543624161


In [186]:
import gensim

summ_gensim = gensim.summarization.summarize(text)
print('gensim')
scores(5, summ_gensim)

gensim
1 0.9615384615384616
2 0.9411764705882353
3 0.7410071942446043
4 0.6258503401360545
5 0.5100671140939598


In [185]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer


def format_sumy(sumy_summary):
    return ''.join(char for sent in sumy_summary for char in str(sent) if char in string.printable)

parser = PlaintextParser(text, Tokenizer('english'))

text_rank = TextRankSummarizer()
summ_text_rank = format_sumy(text_rank(parser.document, 5))
print('sumy - text rank')
scores(5, summ_text_rank)

print()

lex_rank = LexRankSummarizer()
summ_lex_rank = format_sumy(lex_rank(parser.document, 2))
print('sumy - lex rank')
scores(5, summ_lex_rank)

sumy - text rank
1 0.9615384615384616
2 0.8529411764705882
3 0.6330935251798561
4 0.5034013605442177
5 0.4228187919463087

sumy - lex rank
1 0.9230769230769231
2 0.7254901960784313
3 0.48201438848920863
4 0.35374149659863946
5 0.26174496644295303
